In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
# Set headers
heads = requests.utils.default_headers()
heads.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
   })

In [3]:
# helper functions to retrieve detials about each post
def getUser(header):
    user = ''
    user_tag = header.find("a", class_="user")
    if user_tag:
        user_tag.get_text()
    return user

def getTimestamp(header):
    time = ''
    date = ''
    tag_datetime = header.contents[-1]
    if tag_datetime:
        time = tag_datetime.contents[0].contents[0]
    if len(tag_datetime.contents) == 3:
        date = tag_datetime.contents[-1].contents[0]
    elif len(tag_datetime.contents) == 6:
        date = tag_datetime.contents[2].contents[0] + ' ' + tag_datetime.contents[4].contents[0]
    date_time = '{} {}'.format(date, time) 
    return date_time

def getLikesShares(body):
    likes = ''
    shares = ''
    s_class = body.find("p", class_='s')
    if s_class:
        likes = s_class.find_all('b')[0].get_text()
        shares = s_class.find_all('b')[1].get_text()
    return [likes, shares]

def getQuote(body):
    quotes = []
    content = body.find('div', class_='narrow')
    blockquotes = content.find_all('blockquote')
    for blockquote in blockquotes:
        a_tag = blockquote.find('a')
        if a_tag:
            _id = a_tag.get('href')
            quotes.append(_id)
    return quotes

def getText(body):
    content = body.find('div', class_='narrow')
    text = ''
    while content.blockquote:
        content.blockquote.extract()
    text = content.get_text()
    return text

def getPostID(header):
    post_id = ''
    name = header.find_all('a')[0].get('name')
    if name:
        post_id = name
    return post_id

def is_post_equal(post1, post2):
    return post1 == post2

def get_thread(link):
    _id = ''
    name = link.get('name')
    if name not in ('top', None):
        _id = name
    return _id

In [4]:
# retrieve details of each post
def parse_post(header, body):
    post = {}
    post['posted'] = getTimestamp(header)
    post['user'] = getUser(header)
    post['post_id'] = getPostID(header)
    post['text'] = getText(body)
    post['has_quote'] = True if getQuote(body) else False
    post['quotes'] = getQuote(body)
    post['shares'] = getLikesShares(body)[1]
    post['likes'] = getLikesShares(body)[0]
    post['retrieved'] = datetime.now().strftime("%H:%M:%S %d-%m-%Y")
    return post

In [5]:
# retrieve posts from thread
def parse_thread(thread):
    page = 0 # start from the first page
    next_page = True
    index_post = ''
    previous_index_post = ''

    data = []
    #import pdb; pdb.set_trace()
    while next_page:
        start_url = 'https://www.nairaland.com/{}/{}'.format(thread, page)
        r1 = requests.get(start_url, heads)
        thread_html = BeautifulSoup(r1.text, 'lxml')

        headers = thread_html.find_all('td', class_='bold l pu')
        bodys = thread_html.find_all('td', class_='l w pd')

        #retrieve first post in the thread 
        index_post = getPostID(headers[0]) 

        if page > 0:
            # compare first post on current page with previous page
            if is_post_equal(index_post, previous_index_post):
                break
        for i in range(len(headers)):
            header = headers[i]
            body = bodys[i]
            post = parse_post(header, body)
            post.update({'page_no': page, 'thread':thread})
            data.append(post)
        
        previous_index_post = index_post
        page += 1

    print('Thread: {} Page: {} No of Posts {}'.format(thread, page, len(data)))

    return data

In [6]:
def parse_forum(forum):
    start_url = 'https://www.nairaland.com/{}/new'.format(forum)
    r1 = requests.get(start_url, heads)
    raw_html = BeautifulSoup(r1.text, 'html5lib')

    df = pd.DataFrame()

    # retrieve the number of pages in the forum
    page = int(raw_html.select('body > div > p:nth-child(7)')[0].select('b')[1].text) #
    
    for i in tqdm(range(page)):
        next_page = start_url + '{}'.format(i)
        r2 = requests.get(next_page, heads)
        forum_html = BeautifulSoup(r2.text, 'html5lib')
        links = raw_html.find_all('a')
        for link in links:
            thread = get_thread(link)
            if thread:
                res = parse_thread(thread)
                df_res = pd.DataFrame(res)
                df_res['forum'] = forum
                df = df.append(df_res)
    return df

In [7]:
df = parse_forum('health')

0%|          | 0/2 [00:00<?, ?it/s]Thread: 5683160 Page: 1 No of Posts 1
Thread: 5683084 Page: 1 No of Posts 1
Thread: 5683066 Page: 1 No of Posts 1
Thread: 5683047 Page: 1 No of Posts 1
Thread: 5683026 Page: 1 No of Posts 1
Thread: 5683014 Page: 1 No of Posts 1
Thread: 5682985 Page: 1 No of Posts 1
Thread: 5682972 Page: 1 No of Posts 2
Thread: 5682964 Page: 1 No of Posts 1
Thread: 5682948 Page: 1 No of Posts 1
Thread: 5682946 Page: 1 No of Posts 1
Thread: 5682915 Page: 1 No of Posts 3
Thread: 5682891 Page: 1 No of Posts 1
Thread: 5682857 Page: 1 No of Posts 1
Thread: 5682836 Page: 1 No of Posts 2
Thread: 5682825 Page: 1 No of Posts 1
Thread: 5682785 Page: 1 No of Posts 1
Thread: 5682754 Page: 1 No of Posts 1
Thread: 5682742 Page: 1 No of Posts 6
Thread: 5682701 Page: 1 No of Posts 1
Thread: 5682694 Page: 1 No of Posts 1
Thread: 5682681 Page: 1 No of Posts 1
Thread: 5682622 Page: 1 No of Posts 1
Thread: 5682571 Page: 1 No of Posts 3
Thread: 5682556 Page: 1 No of Posts 6
Thread: 5682544

In [10]:
df.to_csv('health_sample.csv')

In [11]:
0%500

0

In [12]:
for i in range(5):
    print(i)

0
1
2
3
4


In [16]:
500%500

1